# Temporal Firing Rate Pattern

##### Reads from many sheets (files), measures per neuron, per bin and per network with frequency domain inlcuded
- Gets the excel out put of neuroexplorer (ratehisto output of nex file)

In [ ]:
import os
import pandas as pd

folder_path = r'C:\FiringFreqHisto'
output_file = r'C:\FiringFreqHisto_02.xlsx'

sheet1_data = {}
sheet2_data = {'Bin': None}
sheet3_data = {'Frequency (Hz)': list(range(101))}
sheet4_data = {'Frequency (Hz)': list(range(101))}

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        excel_file = pd.ExcelFile(file_path)
        for sheet_name in excel_file.sheet_names:
            df = excel_file.parse(sheet_name)
            if 'nr_001' in df.columns:
                df = df.drop(columns=['nr_001'])
            bins = df['Bin Left']
            neuron_data = df.iloc[:, 2:]  
            if neuron_data.empty:
                continue
            neuron_averages = neuron_data.mean(axis=0)
            sheet1_data[f'{file_name}_{sheet_name}'] = neuron_averages
            bin_averages = neuron_data.mean(axis=1)
            if sheet2_data['Bin'] is None:
                sheet2_data['Bin'] = bins 
            sheet2_data[f'{file_name}_{sheet_name}'] = bin_averages.values
            freq_counts = neuron_data.stack().value_counts(bins=range(101), sort=False)
            freq_counts.index = freq_counts.index.left
            freq_counts = freq_counts.reindex(range(101), fill_value=0)
            sheet3_data[f'{file_name}_{sheet_name}'] = freq_counts.values
            total_events = freq_counts.sum()
            freq_percentages = freq_counts / total_events * 100
            sheet4_data[f'{file_name}_{sheet_name}'] = freq_percentages.values
sheet1_df = pd.DataFrame(sheet1_data)
if sheet2_data['Bin'] is None:
    raise ValueError("No valid data was found in any of the sheets.")
sheet2_df = pd.DataFrame(sheet2_data)
sheet3_df = pd.DataFrame(sheet3_data)
sheet4_df = pd.DataFrame(sheet4_data)
with pd.ExcelWriter(output_file) as writer:
    sheet1_df.to_excel(writer, sheet_name='Neuron Averages', index=False)
    sheet2_df.to_excel(writer, sheet_name='Bin Averages', index=False)
    sheet3_df.to_excel(writer, sheet_name='Frequency Counts', index=False)
    sheet4_df.to_excel(writer, sheet_name='Frequency Percentages', index=False)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

folder_path = r'C:\FiringFreqHisto_02'
for filename in os.listdir(folder_path):
    if filename.endswith('Baseline.xlsx'):
        file_path = os.path.join(folder_path, filename)
        excel_file = pd.ExcelFile(file_path)
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            if 'nr_001' in df.columns:
                df = df.drop(columns=['nr_001'])
            bins = df['Bin Left'].values
            neuron_data = df.iloc[:, 2:].values
            bin_avg = np.mean(neuron_data, axis=1)
            plt.figure(figsize=(14, 12))  # Adjusted size to accommodate additional plots
            ax = sns.heatmap(neuron_data / neuron_data.max(axis=0), cmap='viridis', cbar=False) 
            cbar = plt.colorbar(ax.collections[0], ax=ax, orientation='vertical', pad=0.02)
            cbar.set_label('Normalized Firing Frequency')
            plt.suptitle(sheet_name, fontsize=14, fontweight='bold', y=1.02)  # Adjusted title position
            ax_avg_neurons = ax.figure.add_axes([ax.get_position().x0, ax.get_position().y1 + 0.15, ax.get_position().width, 0.06])
            neuron_ids = np.arange(neuron_data.shape[1])
            firing_rates = neuron_data.mean(axis=0)  
            ax_avg_neurons.scatter(neuron_ids, firing_rates, color='black', s=20)  
            ax_avg_neurons.set_xlim(0, len(neuron_ids) - 1)  
            ax_avg_neurons.set_xticks([]) 
            ax_avg_neurons.set_yticks([firing_rates.min(), firing_rates.max()])  
            ax_avg_neurons.set_ylabel('Firing Rate (Hz)')
            ax_avg_neurons.yaxis.set_ticks_position('left')
            ax_avg_bins = ax.figure.add_axes([ax.get_position().x1 + 0.02, ax.get_position().y0, 0.1, ax.get_position().height])
            ax_avg_bins.plot(bin_avg, np.arange(len(bin_avg)), color='black')
            ax_avg_bins.invert_yaxis()
            ax_avg_bins.set_ylim(len(bin_avg) - 1, 0)  # Inverted y-axis with min/max bins
            ax_avg_bins.set_xticks([bin_avg.min(), bin_avg.max()])  # Show min and max on x-axis
            ax_avg_bins.set_yticks([])  # Hide y ticks
            ax_avg_bins.set_xticklabels([f'{bin_avg.min():.2f}', f'{bin_avg.max():.2f}'])  # Label min and max x-axis
            #output_path = os.path.join(folder_path, f"{sheet_name}.pdf")
            #plt.savefig(output_path, format='pdf', bbox_inches='tight')  # Ensure everything fits within the PDF
            plt.show()
            plt.close()

## measure synchronicity based on Bin data

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.signal import correlate
from scipy.spatial.distance import pdist, squareform

folder_path = r'C:\FiringFreqHisto_02'
synchronicity_results = []
for filename in os.listdir(folder_path):
    if filename.endswith('Baseline.xlsx'):
        file_path = os.path.join(folder_path, filename)
        excel_file = pd.ExcelFile(file_path)
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            if 'nr_001' in df.columns:
                df = df.drop(columns=['nr_001'])
            neuron_data = df.iloc[:, 2:].values
            neuron_maxes = neuron_data.max(axis=0)
            neuron_maxes[neuron_maxes == 0] = 1 
            normalized_data = neuron_data / neuron_maxes
            normalized_data = np.nan_to_num(normalized_data, nan=0.0, posinf=0.0, neginf=0.0)
            num_neurons = normalized_data.shape[1]
            correlation_matrix = np.corrcoef(normalized_data.T)
            mean_correlation = np.mean(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)])
            mean_scc = np.mean([pearsonr(normalized_data[:, i], normalized_data[:, j])[0]
                                for i in range(num_neurons)
                                for j in range(i + 1, num_neurons)
                                if not np.any(np.isnan(normalized_data[:, i])) and not np.any(np.isnan(normalized_data[:, j]))])
            cross_corrs = [np.max(correlate(normalized_data[:, i], normalized_data[:, j], mode='valid'))
                           for i in range(num_neurons)
                           for j in range(i + 1, num_neurons)]
            mean_cross_corr = np.mean(cross_corrs)
            distances = pdist(normalized_data.T, metric='euclidean')
            mutual_information = np.mean(distances)
            synchronicity_results.append({
                'Source File': filename,
                'Sheet Name': sheet_name,
                'Mean Correlation': mean_correlation,
                'Mean SCC': mean_scc,
                'Mean Cross-Correlation': mean_cross_corr,
                'Mutual Information': mutual_information
            })

synchronicity_df = pd.DataFrame(synchronicity_results)
output_file = os.path.join(folder_path, 'RateHisto_Synchronicity_Metrics.xlsx')
synchronicity_df.to_excel(output_file, index=False)

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.signal import correlate
from scipy.spatial.distance import pdist, squareform

folder_path = r'C:\FiringFreqHisto_02'
synchronicity_results = []
for filename in os.listdir(folder_path):
    if filename.endswith('Baseline.xlsx'):
        file_path = os.path.join(folder_path, filename)
        excel_file = pd.ExcelFile(file_path)
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            if 'nr_001' in df.columns:
                df = df.drop(columns=['nr_001'])
            neuron_data = df.iloc[:, 2:].values
            neuron_maxes = neuron_data.max(axis=0)
            neuron_maxes[neuron_maxes == 0] = 1  
            normalized_data = neuron_data / neuron_maxes
            normalized_data = np.nan_to_num(normalized_data, nan=0.0, posinf=0.0, neginf=0.0)
            num_neurons = normalized_data.shape[1]
            correlation_matrix = np.corrcoef(normalized_data.T)
            mean_correlation = np.mean(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)])
            mean_scc = np.mean([pearsonr(normalized_data[:, i], normalized_data[:, j])[0]
                                for i in range(num_neurons)
                                for j in range(i + 1, num_neurons)
                                if not np.any(np.isnan(normalized_data[:, i])) and not np.any(np.isnan(normalized_data[:, j]))])
            cross_corrs = [np.max(correlate(normalized_data[:, i], normalized_data[:, j], mode='valid'))
                           for i in range(num_neurons)
                           for j in range(i + 1, num_neurons)]
            mean_cross_corr = np.mean(cross_corrs)
            distances = pdist(normalized_data.T, metric='euclidean')
            mutual_information = np.mean(distances)
            pairwise_correlations = [pearsonr(normalized_data[:, i], normalized_data[:, j])[0]
                                     for i in range(num_neurons)
                                     for j in range(i + 1, num_neurons)]
            mean_sync_index = np.mean(pairwise_correlations)
            synchronicity_results.append({
                'Source File': filename,
                'Sheet Name': sheet_name,
                'Mean Correlation': mean_correlation,
                'Mean SCC': mean_scc,
                'Mean Cross-Correlation': mean_cross_corr,
                'Mutual Information': mutual_information,
                'Synchrony Index': mean_sync_index,
                'Number of Neurons': num_neurons
            })

synchronicity_df = pd.DataFrame(synchronicity_results)
output_file = os.path.join(folder_path, 'RateHisto_Synchronicity_Metrics.xlsx')
synchronicity_df.to_excel(output_file, index=False)

RateHisto_Organized

In [ ]:
import pandas as pd

file_path = r'C:\RateHisto_02.xlsx'
xls = pd.ExcelFile(file_path)

sheets_to_process = ['Frequency Percentages', 'Frequency Counts']

processed_dfs = []
for sheet_name in sheets_to_process:
    df = pd.read_excel(xls, sheet_name)
    frequency_values = df['Frequency (Hz)']
    data = []
    for col in df.columns[1:]:
        sample_name = col
        group = 'Vasc' if 'VASCU' in sample_name else 'NonVasc'
        week = 'w17' if 'w17' in sample_name else 'w20' if 'w20' in sample_name else 'w23'
        sample_values = df[col].values
        row = [group, week, sample_name] + sample_values.tolist()
        data.append(row)
    columns = ['Group', 'Week', 'Sample Name'] + frequency_values.tolist()
    processed_df = pd.DataFrame(data, columns=columns)
    processed_dfs.append((sheet_name, processed_df))
output_file_path = r'C:\RateHisto_Organized.xlsx'

with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    for sheet_name, processed_df in processed_dfs:
        processed_df.to_excel(writer, sheet_name=sheet_name, index=False)

## Plot Percentage per Freq Domain

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

file_path = r'C:\RateHisto_Organized.xlsx'
df = pd.read_excel(file_path, sheet_name='Frequency Percentages')

df_melted = df.melt(id_vars=['Group', 'Week', 'Sample Name'], var_name='Frequency (Hz)', value_name='Percentage')
df_melted['Frequency (Hz)'] = pd.to_numeric(df_melted['Frequency (Hz)'])

grouped = df_melted.groupby(['Group', 'Week', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sem_percentage=('Percentage', 'sem')
).reset_index()

weeks = grouped['Week'].unique()
fig, axes = plt.subplots(1, len(weeks), figsize=(20, 6), sharey=True)

for i, week in enumerate(weeks):
    ax = axes[i]
    sns.lineplot(
        data=grouped[grouped['Week'] == week], 
        x='Frequency (Hz)', 
        y='mean_percentage', 
        hue='Group', 
        style='Group', 
        markers=True, 
        errorbar='se',  # Updated errorbar parameter
        ax=ax, 
        palette='Set1'
    )
    ax.set_title(f'Week {week}')
    ax.set_xlabel('Frequency (Hz)')
    if i == 0:
        ax.set_ylabel('Average Percentage')
    else:
        ax.set_ylabel('')
    ax.legend(title='Group')

plt.tight_layout()
plt.show()

grouped_all_weeks = df_melted.groupby(['Group', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sem_percentage=('Percentage', 'sem')
).reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=grouped_all_weeks, 
    x='Frequency (Hz)', 
    y='mean_percentage', 
    hue='Group', 
    style='Group', 
    markers=True, 
    errorbar='se',  
    palette='Set1'
)
plt.title('Mean Percentage Across All Weeks')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Average Percentage')
plt.legend(title='Group')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Excel file and read the Frequency Percentages sheet
file_path = r'C:\RateHisto_Organized.xlsx'
df = pd.read_excel(file_path, sheet_name='Frequency Percentages')
df_melted = df.melt(id_vars=['Group', 'Week', 'Sample Name'], var_name='Frequency (Hz)', value_name='Percentage')
df_melted['Frequency (Hz)'] = pd.to_numeric(df_melted['Frequency (Hz)'])
df_melted = df_melted[(df_melted['Frequency (Hz)'] != 0) & (df_melted['Frequency (Hz)'] != 100)]
grouped = df_melted.groupby(['Group', 'Week', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sd_percentage=('Percentage', 'std')  
).reset_index()

custom_palette = {'NonVasc': 'gray', 'Vasc': 'lightcoral'}
weeks = grouped['Week'].unique()
fig, axes = plt.subplots(1, len(weeks), figsize=(20, 6), sharey=True)

for i, week in enumerate(weeks):
    ax = axes[i]
    sns.lineplot(
        data=grouped[grouped['Week'] == week], 
        x='Frequency (Hz)', 
        y='mean_percentage', 
        hue='Group', 
        style='Group', 
        markers=True, 
        ax=ax, 
        palette=custom_palette  # Apply custom palette
    )
    
    for group in grouped['Group'].unique():
        subset = grouped[(grouped['Week'] == week) & (grouped['Group'] == group)]
        ax.errorbar(
            x=subset['Frequency (Hz)'],
            y=subset['mean_percentage'],
            yerr=subset['sd_percentage'],
            fmt='o',  # Marker style
            color=custom_palette[group],  
            capsize=5,  
            alpha=0.5 
        )
    
    ax.set_yscale('log')  
    ax.set_ylim([1e-4, 1e2])  
    ax.set_title(f'Week {week}')
    ax.set_xlabel('Frequency (Hz)')
    if i == 0:
        ax.set_ylabel('Average Percentage')
    else:
        ax.set_ylabel('')
    ax.legend(title='Group')

plt.tight_layout()
plt.show()

grouped_all_weeks = df_melted.groupby(['Group', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sd_percentage=('Percentage', 'std') 
).reset_index()

plt.figure(figsize=(4, 6))
sns.lineplot(
    data=grouped_all_weeks, 
    x='Frequency (Hz)', 
    y='mean_percentage', 
    hue='Group', 
    style='Group', 
    markers=True, 
    ax=plt.gca(), 
    palette=custom_palette  
)

for group in grouped_all_weeks['Group'].unique():
    subset = grouped_all_weeks[grouped_all_weeks['Group'] == group]
    plt.errorbar(
        x=subset['Frequency (Hz)'],
        y=subset['mean_percentage'],
        yerr=subset['sd_percentage'],
        fmt='o',  # Marker style
        color=custom_palette[group], 
        capsize=5,  
        alpha=0.5  
    )

plt.yscale('log')  
plt.ylim([1e-4, 1e2])  
plt.title('Mean Percentage Across All Weeks')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Average Percentage')
plt.legend(title='Group')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

file_path = r'C:\RateHisto_Organized.xlsx'
df = pd.read_excel(file_path, sheet_name='Frequency Percentages')
df_melted = df.melt(id_vars=['Group', 'Week', 'Sample Name'], var_name='Frequency (Hz)', value_name='Percentage')
df_melted['Frequency (Hz)'] = pd.to_numeric(df_melted['Frequency (Hz)'])
df_melted = df_melted[(df_melted['Frequency (Hz)'] != 0) & (df_melted['Frequency (Hz)'] != 100)]
grouped = df_melted.groupby(['Group', 'Week', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sd_percentage=('Percentage', 'std')  
).reset_index()

custom_palette = {'NonVasc': 'gray', 'Vasc': 'lightcoral'}
marker_properties = {
    'NonVasc': {'size': 3},
    'Vasc': {'size': 3}
}

weeks = grouped['Week'].unique()
fig, axes = plt.subplots(1, len(weeks), figsize=(10, 6), sharey=True)

for i, week in enumerate(weeks):
    ax = axes[i]
    for group in grouped['Group'].unique():
        subset = grouped[(grouped['Week'] == week) & (grouped['Group'] == group)]
        properties = marker_properties[group]
        sns.lineplot(
            data=subset, 
            x='Frequency (Hz)', 
            y='mean_percentage', 
            hue='Group', 
            style='Group', 
            markers='o',  
            ax=ax, 
            palette=custom_palette  
        )
        
        # Plot upper error bars
        upper_error = subset['sd_percentage']  
        ax.errorbar(
            x=subset['Frequency (Hz)'],
            y=subset['mean_percentage'],
            yerr=[np.zeros_like(upper_error), upper_error],  
            fmt='o',  
            color=custom_palette[group],  
            capsize=3,  
            alpha=0.3,  
            #markeredgecolor=properties['edgecolor'],  
            markersize=properties['size']  
        )
    
    ax.set_yscale('log') 
    ax.set_ylim([1e-3, 1e2]) 
    ax.set_title(f'Week {week}')
    ax.set_xlabel('Frequency (Hz)')
    if i == 0:
        ax.set_ylabel('Average Percentage')
    else:
        ax.set_ylabel('')
    ax.legend(title='Group')

plt.tight_layout()
plt.show()

grouped_all_weeks = df_melted.groupby(['Group', 'Frequency (Hz)']).agg(
    mean_percentage=('Percentage', 'mean'),
    sd_percentage=('Percentage', 'std')  
).reset_index()

plt.figure(figsize=(4, 6))
for group in grouped_all_weeks['Group'].unique():
    subset = grouped_all_weeks[grouped_all_weeks['Group'] == group]
    properties = marker_properties[group]
    sns.lineplot(
        data=subset, 
        x='Frequency (Hz)', 
        y='mean_percentage', 
        hue='Group', 
        style='Group', 
        markers='o',  
        ax=plt.gca(), 
        palette=custom_palette  
    )
    
    # Plot upper error bars
    upper_error = subset['sd_percentage']  
    plt.errorbar(
        x=subset['Frequency (Hz)'],
        y=subset['mean_percentage'],
        yerr=[np.zeros_like(upper_error), upper_error],  
        fmt='o',  
        color=custom_palette[group],  
        capsize=3,  
        alpha=0.3,  
        #markeredgecolor=properties['edgecolor'],  
        markersize=properties['size']  
    )

plt.yscale('log')  
plt.ylim([1e-3, 1e2])  
plt.title('Mean Percentage Across All Weeks')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Average Percentage')
plt.legend(title='Group')
plt.tight_layout()
plt.show()